In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats as st
import seaborn as sns
import time
from sklearn.model_selection import KFold

%matplotlib inline

In [ ]:
# Function to generate the feature matrix using feature mapping
def phi_x(omega_phi,b_phi,x_phi,M_phi):
    return (np.sqrt(2) / np.sqrt(M_phi)) * np.cos(np.outer(omega_phi, x_phi) + b_phi[:, np.newaxis])

# Function representing the true underlying relationship
def f_true(x_f):
    return 0.8 * np.sin(np.pi*x_f)

def generate_time_series(n_ge):
    x_ge = np.zeros(n_ge)
    for t in range(1, n_ge):
        x_ge[t] = f_true(x_ge[t-1]) + np.random.uniform(-0.6, 0.6)
    return x_ge

def regularized_regression(S_M_re, y_re, lam_re,M_re):
    n_re= y_re.shape[0]
    theta_re = np.linalg.solve(np.dot(S_M_re, S_M_re.T) + lam_re * n_re * np.identity(M_re), np.dot(S_M_re, y_re))
    return theta_re


def cross_validate_lam(X, y,omega,b,M, lam_values, num_folds=5, random_state=None):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    mse_values = []

    for lam in lam_values:
        fold_mse = []

        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            S_M_train = phi_x(omega,b,X_train,M)
            if S_M_train.shape[1]==1:
                S_M_train.reshape(S_M_train.shape[0],)
            
            theta = regularized_regression(S_M_train, y_train, lam,M)

            S_M_val = phi_x(omega,b,X_val,M)
            pred = np.dot(S_M_val.T, theta)

            fold_mse.append(mean_squared_error(f_true(X_val), pred))

        mse_values.append(np.mean(fold_mse))

    optimal_lam = lam_values[np.argmin(mse_values)]
    return optimal_lam

# Define a function to encapsulate the loop
def run_optimization_loop(n_points_values, ratio, num_iterations, train_percentage=0.6):
    mse_results = {}
    time_results = {}

    for n_points in n_points_values:
        mse_optimal_list = []
        time_list = []
        train_size = int(train_percentage * n_points)
        M = int(ratio*train_size)
        print(M)

        for _ in range(num_iterations):
            # Generate time series data
            time_series = generate_time_series(n_points)
            
            # Use only the first train_percentage of data for training
            
            x_train = time_series[:train_size-1]
            y_train = time_series[1:train_size]
            
            # The rest is used for testing
            x_test = time_series[train_size-1:]

            # Generate random parameters
            omega_1 = np.random.normal(0, np.sqrt(2), M)
            b_1 = np.random.uniform(0, 2 * np.pi, M)

            lam_values = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
            start_time = time.time()
            optimal_lam_1= cross_validate_lam(X=x_train, y=y_train,omega=omega_1,b=b_1,M=M, lam_values=lam_values, num_folds=3)

            # Now use the optimal lambda for the final model
            S_M = phi_x(omega_1,b_1,x_train,M)
            theta_optimal = regularized_regression(S_M, y_train, optimal_lam_1,M)

            # Make predictions using the trained model with optimal lambda on the test data
            pred_optimal = np.dot(phi_x(omega_1,b_1,x_test,M).T, theta_optimal)

            # Calculate mean squared error on the test data
            mse_optimal = mean_squared_error(f_true(x_test), pred_optimal)

            end_time = time.time()

            # Append results to lists
            mse_optimal_list.append(mse_optimal)
            time_list.append(end_time - start_time)

        mse_results[n_points] = mse_optimal_list
        time_results[n_points] = time_list

    return mse_results, time_results

In [ ]:
def run_optimization_loop_vary_ra(n, ratio_values, num_iterations, train_percentage=0.6):
    mse_results = {}
    time_results = {}

    for ratio in ratio_values:
        mse_optimal_list = []
        time_list = []
        train_size = int(train_percentage * n)
        M = int(ratio * train_size)
        print(M)

        for _ in range(num_iterations):
            # Generate time series data
            time_series = generate_time_series(n)
            
            # Use only the first train_percentage of data for training
            x_train = time_series[:train_size-1]
            y_train = time_series[1:train_size]
            
            # The rest is used for testing
            x_test = time_series[train_size-1:]

            # Generate random parameters
            omega_1 = np.random.normal(0, np.sqrt(2), M)
            b_1 = np.random.uniform(0, 2 * np.pi, M)

            lam_values = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
            start_time = time.time()
            optimal_lam_1 = cross_validate_lam(X=x_train, y=y_train, omega=omega_1, b=b_1, M=M, lam_values=lam_values, num_folds=3)

            # Now use the optimal lambda for the final model
            S_M = phi_x(omega_1, b_1, x_train, M)
            theta_optimal = regularized_regression(S_M, y_train, optimal_lam_1, M)

            # Make predictions using the trained model with optimal lambda on the test data
            pred_optimal = np.dot(phi_x(omega_1, b_1, x_test, M).T, theta_optimal)

            # Calculate mean squared error on the test data
            mse_optimal = mean_squared_error(f_true(x_test), pred_optimal)

            end_time = time.time()

            # Append results to lists
            mse_optimal_list.append(mse_optimal)
            time_list.append(end_time - start_time)

        mse_results[ratio] = mse_optimal_list
        time_results[ratio] = time_list

    return mse_results, time_results

# Vary n

In [ ]:
# Define values for n_points and run the loop
n_points_values =[1500,3000,4500,7500,11250,15000]

num_iterations = 100

ratio= 0.001 #0.005,0.01,0.05,0.1
mse_results_n_points, time_results_n_points= run_optimization_loop(n_points_values, ratio, num_iterations, train_percentage=2/3)

# Vary M

In [ ]:
# Define values for n_points and run the loop
n = 6000

num_iterations = 100

ratio_values = np.exp(np.linspace(-7,-1,15))
mse_results_n_points_1, time_results_n_points_1= run_optimization_loop_vary_ra(n, ratio_values, num_iterations, train_percentage=2/3)